# Spaceship Lightgbm Submission

In [ ]:
!pip install lightgbm==4.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from lightgbm import LGBMClassifier

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier

# Load datasets
X_df = pd.read_csv('spaceship_train_X_v2.csv')
y_df = pd.read_csv('spaceship_train_y.csv')
X_submission = pd.read_csv('spaceship_test_X_v2.csv')

# Drop unnecessary columns
y_df.drop('Unnamed: 0', axis=1, inplace=True)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df['Transported'], stratify=y_df['Transported'], test_size=0.1, train_size=0.9, random_state=1)

# Standardize data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_submission_scaled = scaler.transform(X_submission)

# Define base models
rf = RandomForestClassifier(random_state=42)
gb = GradientBoostingClassifier(random_state=42)
lgbm = LGBMClassifier(random_state=42)

# Create the voting ensemble
voting_model = VotingClassifier(
    estimators=[
        ('rf', rf),
        ('gb', gb),
        ('lgbm', lgbm)
    ],
    voting='soft'  # 'soft' uses the predicted probabilities to average the predictions
)

# Define parameter grid for grid search
param_grid = {
    'rf__n_estimators': [100],
    'rf__max_depth': [10],
    'gb__n_estimators': [100],
    'gb__learning_rate': [0.01],
    'lgbm__n_estimators': [100],
    'lgbm__max_depth': [10],
    'lgbm__learning_rate': [0.01]
}

# Create grid search
grid_search = GridSearchCV(estimator=voting_model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

# Train the ensemble model
grid_search.fit(X_train_scaled, y_train)

y_pred_light = grid_search.predict(X_test_scaled)
print("Best parameters:", grid_search.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred_light))
print("Classification Report:\n", classification_report(y_test, y_pred_light))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_light))

# Predict on test set
y_test_pred = grid_search.predict(X_test_scaled)
y_submission_pred = grid_search.predict(X_submission_scaled)

# Create submission file
submission_dict = {'PassengerId': X_submission['PassengerId'], 'Transported': y_submission_pred.astype('bool')}
submission_df = pd.DataFrame(submission_dict)
submission_df.to_csv('spaceship_voting_ensemble.csv', index=False)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 3152, number of negative: 3106
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001665 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2156
[LightGBM] [Info] Number of data points in the train set: 6258, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503675 -> initscore=0.014701
[LightGBM] [Info] Start training from score 0.014701
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't

In [ ]:
#voting classifier
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load datasets
X_df = pd.read_csv('spaceship_train_X_v2.csv')
y_df = pd.read_csv('spaceship_train_y.csv')
X_submission = pd.read_csv('spaceship_test_X_v2.csv')

# Drop unnecessary columns
y_df.drop('Unnamed: 0', axis=1, inplace=True)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df['Transported'], stratify=y_df['Transported'], test_size=0.1, train_size=0.9, random_state=1)

# Standardize data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_submission_scaled = scaler.transform(X_submission)

# Define base models
rf = RandomForestClassifier(random_state=42)
gb = GradientBoostingClassifier(random_state=42)
lgbm = LGBMClassifier(random_state=42)

# Define meta model
meta_model = LogisticRegression()

# Create the stacking ensemble
stacking_model = StackingClassifier(
    estimators=[
        ('rf', rf),
        ('gb', gb),
        ('lgbm', lgbm)
    ],
    final_estimator=meta_model,
    cv=5
)

# Define parameter grid for grid search
param_grid = {
    'rf__n_estimators': [200],
    'rf__max_depth': [20],
    'gb__n_estimators': [200],
    'gb__learning_rate': [0.1],
    'gb__max_depth': [5],
    'lgbm__n_estimators': [200],
    'lgbm__learning_rate': [0.01],
    'lgbm__max_depth': [10]
}


# Menggunakan StratifiedKFold untuk handling imbalanced data
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Create grid search
grid_search = GridSearchCV(estimator=stacking_model, param_grid=param_grid, cv=cv, n_jobs=-1, scoring='accuracy')

# Train the ensemble model
grid_search.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred_light = grid_search.predict(X_test_scaled)
print("Best parameters:", grid_search.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred_light))
print("Classification Report:\n", classification_report(y_test, y_pred_light))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_light))

# Predict on test set
y_submission_pred = grid_search.predict(X_submission_scaled)

# Create submission file
submission_dict = {'PassengerId': X_submission['PassengerId'], 'Transported': y_submission_pred.astype('bool')}
submission_df = pd.DataFrame(submission_dict)
submission_df.to_csv('spaceship_stacking_ensemble_improved.csv', index=False)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 3546, number of negative: 3494
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001943 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2156
[LightGBM] [Info] Number of data points in the train set: 7040, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503693 -> initscore=0.014773
[LightGBM] [Info] Start training from score 0.014773
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't

In [ ]:
!pip install optuna==3.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 69.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
submission_df.to_csv('spaceship_ensemble_optuna.csv', index=False)

# Bring the Data In

In [ ]:
X_df = pd.read_csv('spaceship_train_X_v2.csv')
y_df = pd.read_csv('spaceship_train_y.csv')
X_submission = pd.read_csv('spaceship_test_X_v2.csv')

y_df.drop('Unnamed: 0', axis=1, inplace=True)
X_df.head()

,PassengerId,CryoSleep,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Expenditure,NoSpending,...,CabinSide_Z,CabinDeck_A,CabinDeck_B,CabinDeck_C,CabinDeck_D,CabinDeck_E,CabinDeck_F,CabinDeck_G,CabinDeck_T,AgeEncoded
0,0001_01,0,0,-0.337530,-0.283865,-0.287383,-0.273826,-0.265831,-0.518357,1,...,0,0,1,0,0,0,0,0,0,3.0
1,0002_01,0,0,-0.172455,-0.278226,-0.245562,0.213965,-0.227033,-0.256582,0,...,0,0,0,0,0,0,1,0,0,2.0
2,0003_01,0,1,-0.272409,1.956643,-0.287383,5.692512,-0.222625,3.174596,0,...,0,1,0,0,0,0,0,0,0,4.0
3,0003_02,0,0,-0.337530,0.519986,0.333240,2.684020,-0.095651,1.322607,0,...,0,1,0,0,0,0,0,0,0,2.0
4,0004_01,0,0,0.121349,-0.240007,-0.034784,0.228181,-0.264067,-0.130318,0,...,0,0,0,0,0,0,1,0,0,1.0


In [ ]:
#Standarisasi data
scaler = StandardScaler()
X_df_scaled = scaler.fit_transform(X_df)
X_submission_scaled = scaler.transform(X_submission)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df_scaled, y_df, stratify=y_df, test_size=0.1, train_size=0.9, random_state=1)

# Mengubah target menjadi 1D array
y_train = y_train['Transported'].values
y_test = y_test['Transported'].values

# Model Training

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LGBMClassifier(random_state=42))
])



param_grid = {
    'classifier__n_estimators': [100, 300, 500],
    'classifier__max_depth': [3, 6, 10],
    'classifier__learning_rate': [0.01, 0.1],
    'classifier__subsample': [0.8, 1],
    'classifier__colsample_bytree': [0.8, 1]
}

In [ ]:

# Membuat GridSearchCV
grid_search = GridSearchCV(pipeline,param_grid, cv=5, scoring='accuracy', verbose=1)
grid_search.fit(X_train, y_train)

# Menampilkan parameter terbaik dan skor
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [ ]:
best_light = grid_search.best_estimator_
y_pred_light = best_light.predict(X_test)
print("Best parameters:", grid_search.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred_light))
print("Classification Report:\n", classification_report(y_test, y_pred_light))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_light))

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Best parameters: {'classifier__colsample_bytree': 0.8, 'classifier__learning_rate': 0.01, 'classifier__max_depth': 10, 'classifier__n_estimators': 500, 'classifier__subsample': 0.8}
Accuracy: 0.8091954022988506
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.77      0.80       432
           1       0.79      0.84      0.82       438

    accuracy                           0.81       870
   macro avg       0.81      0.81      0.81       870
weighted avg       0.81      0.81      0.81       870

Confusion Matrix:
 [[334  98]
 [ 68 370]]


In [ ]:
y_pred = grid_search.predict(X_test)
print("Best parameters:", grid_search.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Best parameters: {'classifier__colsample_bytree': 0.8, 'classifier__learning_rate': 0.01, 'classifier__max_depth': 10, 'classifier__n_estimators': 500, 'classifier__subsample': 0.8}
Accuracy: 0.8091954022988506
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.77      0.80       432
           1       0.79      0.84      0.82       438

    accuracy                           0.81       870
   macro avg       0.81      0.81      0.81       870
weighted avg       0.81      0.81      0.81       870

Confusion Matrix:
 [[334  98]
 [ 68 370]]


# Submission Prediction

In [ ]:
# For training, we use ALL data from spaceship_train_X_v2.csv and spaceship_train_y.csv
grid_search.fit(X_df, y_df['Transported'].values)
# Prediksi data submission
y_prediction = grid_search.predict(X_submission)
print(y_prediction)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [ ]:
submission_dict = {'PassengerId':X_submission['PassengerId'], 'Transported':y_prediction.astype('bool')}
submission_dict = pd.DataFrame(submission_dict)
submission_dict

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


## Export CSV

In [ ]:
submission_dict.to_csv('spaceship_lightgbm2.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2a79941c-6614-47fe-9427-0e9f23998893' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>